In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
clean_data= '/content/drive/MyDrive/Colab Notebooks/clean_dataset_oversample'
with open(clean_data, "r") as file:
    lines = file.readlines()
    
clean_full = [line.split() for line in lines]

In [3]:
import random
random.seed(3)
#on travaille avec un échantillon de 100000 tweets
clean_sample = random.sample(clean_full, 100000)
print(len(clean_sample))

100000


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [5]:
#utiliser la méthode de BOW avec:
""" le cleaning étant déjà fait on utilise directement les données"""
bow_vect = CountVectorizer()
bow = bow_vect.fit_transform([' '.join(tweet) for tweet in clean_sample])

In [6]:
#utiliser la méthode de Tfidfvectorizer avec:
""" le cleaning étant déjà fait on uilise directement les données"""
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([' '.join(tweet) for tweet in clean_sample])

In [7]:
# Parameters tuning using Grid Search
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

In [8]:
%%time

grid_params = {'n_components' : list(range(5,10))}
# LDA model
lda = LatentDirichletAllocation()
lda_model = GridSearchCV(lda,param_grid=grid_params)
lda_model.fit(bow)
# Estimators for LDA model
lda_model_bow = lda_model.best_estimator_


CPU times: user 1h 8min 23s, sys: 14.4 s, total: 1h 8min 38s
Wall time: 1h 8min 34s


In [9]:
print("Best LDA model's params" , lda_model.best_params_)
print("Best log likelihood Score for the LDA model",lda_model.best_score_)
print("LDA model Perplexity on train data", lda_model_bow.perplexity(bow))

Best LDA model's params {'n_components': 5}
Best log likelihood Score for the LDA model -1529094.578416862
LDA model Perplexity on train data 14080.92471291878


In [ ]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [11]:
pyLDAvis.sklearn.prepare(lda_model_bow,bow, bow_vect ,mds='tsne')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:827: FutureWarning: 'square_distan

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      150.222153  -72.269554       1        1  21.940522
3       51.163364 -159.968018       2        1  20.526574
2      -73.144554   21.523815       3        1  19.962335
4      -66.410133 -110.606438       4        1  19.007852
1       54.417324   12.443444       5        1  18.562717, topic_info=           Term         Freq        Total Category  logprob  loglift
65141      sale  2730.000000  2730.000000  Default  30.0000  30.0000
59398      pute  1915.000000  1915.000000  Default  29.0000  29.0000
31469      gros  1330.000000  1330.000000  Default  28.0000  28.0000
7902       bien  3739.000000  3739.000000  Default  27.0000  27.0000
27425      fils   939.000000   939.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
75849     venir   328.136448  1440.926753   Topic5  -5.9412   0.2044
33266  histoire   281.218336   719.298156   Topic5  -6.0955   0.7449
7902       bien   339.698889  3739.098565   Topic5  -5.9065  -0.7145
26416   falloir   297.409369  2153.870682   Topic5  -6.0395  -0.2959
76859      voir   284.254280  3712.690924   Topic5  -6.0847  -0.8856

[398 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
550        1  0.172455    accord
550        2  0.009322    accord
550        3  0.151480    accord
550        4  0.666514    accord
914        5  0.979835     adele
...      ...       ...       ...
80109      1  0.005175    écoute
80109      2  0.150085    écoute
80109      4  0.010351    écoute
80109      5  0.838403    écoute
80519      3  0.992628  épidémie

[824 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 5, 2])

On retrouve 5 clusters : 

1- haine
2- politique
3- santé
4- et 5- ???

In [ ]:
%%time
# Parameters tuning using Grid Search
grid_params = {'n_components' : list(range(5,10))}
# LDA model
lda = LatentDirichletAllocation()
lda_model = GridSearchCV(lda,param_grid=grid_params)
lda_model.fit(X)
# Estimators for LDA model
lda_model1 = lda_model.best_estimator_

CPU times: user 2min 30s, sys: 855 ms, total: 2min 31s
Wall time: 2min 44s


In [ ]:
print("Best LDA model's params" , lda_model.best_params_)
print("Best log likelihood Score for the LDA model",lda_model.best_score_)
print("LDA model Perplexity on train data", lda_model1.perplexity(X))

Best LDA model's params {'n_components': 5}
Best log likelihood Score for the LDA model -37146.70313499427
LDA model Perplexity on train data 34847.91595174759


In [ ]:
pyLDAvis.sklearn.prepare(lda_model1, X,vectorizer,mds='tsne')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:827: FutureWarning: 'square_distan

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      150.222153  -72.269554       1        1  23.021954
4       51.163364 -159.968018       2        1  22.647206
3      -73.144554   21.523815       3        1  20.240495
0      -66.410133 -110.606438       4        1  17.411003
1       54.417324   12.443444       5        1  16.679342, topic_info=            Term       Freq      Total Category  logprob  loglift
8255   président   8.000000   8.000000  Default  30.0000  30.0000
9311      savoir  20.000000  20.000000  Default  29.0000  29.0000
6185       maire   8.000000   8.000000  Default  28.0000  28.0000
251          aim   4.000000   4.000000  Default  27.0000  27.0000
253         aime   9.000000   9.000000  Default  26.0000  26.0000
...          ...        ...        ...      ...      ...      ...
329        aller   2.927514  37.490561   Topic5  -6.5767  -0.7589
10952       voir   2.445024  20.818391   Topic5  -6.7568  -0.3508
6372       matin   2.099160   5.403420   Topic5  -6.9093   0.8455
7610        pays   2.129136   7.604306   Topic5  -6.8951   0.5180
10954       vois   2.070472   5.747720   Topic5  -6.9231   0.7700

[349 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
75         1  0.142351  accord
75         3  0.142351  accord
75         4  0.142351  accord
75         5  0.427052  accord
162        2  0.266585   adore
...      ...       ...     ...
11502      2  0.358789    être
11502      3  0.186570    être
11502      4  0.136340    être
11502      5  0.114812    être
11532      4  0.658601   𝕟𝕠𝕞𝕚𝕟

[575 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 4, 1, 2])